In [1]:
import gymnasium as gym


ModuleNotFoundError: No module named 'gymnasium'

In [ ]:
from gym import spaces
import numpy as np
import scanpy as sc
import svae
import cellrank as cr


In [2]:
import numpy as np

In [1]:
import scanpy as sc

ModuleNotFoundError: No module named 'scanpy'

In [8]:
class EmbeddingEnv(gym.Env):
    def __init__(self, embedding_file, transition_matrix):
        # Load the embedding from the file
        self.embedding = np.load(embedding_file) # must be able to load csv
        
        #defining the observation space
        self.observation_space = spaces.Box(low = -1, high = 1, shape=(self.embedding.shape[1],), dtype=np.float32)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(self.embedding.shape[1],), dtype=np.float32)
        
        # Initialize the state of the environment to a random cell in the embedding
        self.current_observation = self.observation_space.sample()
        
        self.reward_range = (0,1)
        
        
    def step(self, action):
        
        # Apply the modulations to the latent dimensions 
        modulated_embedding = self.embedding + action
        
        # Compute the new position of the cell in the embedding
        distances = np.linalg.norm(modulated_embedding - self.current_observation, axis=1)
        new_observation = modulated_embedding[np.argmin(distances)]
        
        # Calculate the reward based on the probability of absorption
        reward = self.transition_matrix[self.current_observation, new_observation]
        
        # Check if the next state is the terminal state
        done = (new_observation == self.transition_matrix.shape[0] - 1)
        
        # Update the state of the environment 
        self.current_observation = new_observation
        
        # executing the time step in the environment based on the given action
        return self.current_observation, reward, done, {}
    
        
    def reset(self):
        # resetting the state of the environment to a random cell in the embedding
        self.current_observation = self.observation_space.sample()
        return self.current_observation
    
    

In [3]:
class Agent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha  # learning rate
        self.gamma = gamma  # discount factor
        self.epsilon = epsilon  # exploration rate
        self.Q = np.zeros((11, 2))
        
    def choose_action(self, state):
        if np.random.uniform() < self.epsilon:
            action = np.random.choice([-1, 1])
        else:
            action = np.argmax(self.Q[state])
        return action
    
    def learn(self, state, action, reward, next_state, done):
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.Q[next_state])
            self.Q[state, action] += self.alpha * (target - self.Q[state, action])

In [9]:
# Creating the environment based on the class that I built
env = EmbeddingEnv(embedding_file = "/Users/brendamelano/Desktop/svae_latent_array_152.csv", transition_matrix = "/Users/brendamelano/Desktop/svae_abs_probs_152.csv")



OSError: Failed to interpret file '/Users/brendamelano/Desktop/svae_latent_array_152.csv' as a pickle

In [ ]:
# Training the agent with DQN
from stable_baseline3 import DQN 

# Define the Q-learning agent
q_agent = DQN('M1pPolicy', env, learning_rate = 0.001, buffer_size = 50000, verbose = 1)


# Train the agent 
total_timesteps = 50000
q_agent.learn(total_timesteps = total_steps)

# Evaluate the agent 
obs = env.reset()
for i in range(10):
    done = False
    while not done:
        action, _states = q_agent.predict(obs, deterministic = True)
        obs, rewardm done, info = env.step(action)
    print(f"Episode {i+1} finished with reward {reward}")



In [8]:

env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(1000):
   action = env.action_space.sample()  # this is where you would insert your policy
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()
env.close()

In [ ]:
# building regulatory network with Bayesian Network

In [ ]:
# inferring profiles with GRN

In [ ]:
# feeding profiles into cell rank to get transition matrices

# From other script

In [ ]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
class LatentSpaceEnv(gym.Env):
    def __init__(self, action_df, target_point):
        super(LatentSpaceEnv, self).__init__()
        self.action_df = action_df
        self.action_space = gym.spaces.Discrete(len(action_df))
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(128,), dtype=np.float32)
        self.current_state = np.random.rand(128)
        self.target_point = target_point

    def step(self, action):
        # Apply action to current_state
        action_effect = self.action_df.iloc[action].values
        self.current_state += action_effect

        # Calculate Euclidean distance to target_point as the reward
        distance = np.linalg.norm(self.current_state - self.target_point)
        reward = -distance  # Negative reward, lower distance is better

        # Check if the target is reached or sufficiently close
        done = distance < 0.01

        return self.current_state, reward, done, {}

    def reset(self):
        self.current_state = np.random.rand(128)
        return self.current_state

In [ ]:
class SimpleDQN:
    def __init__(self, observation_space, action_space):
        self.observation_space = observation_space
        self.action_space = action_space
        self.model = self.create_model()

    def create_model(self):
        model = tf.keras.Sequential()
        model.add(layers.Input(shape=(self.observation_space.shape[0],)))
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(self.action_space.n, activation='linear'))
        model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')
        return model

    def choose_action(self, state):
        state = np.reshape(state, [1, self.observation_space.shape[0]])
        action_values = self.model.predict(state)
        return np.argmax(action_values[0])

    def learn(self, state, action, reward, next_state):
        # Implement learning process
        pass

In [ ]:
# Initialize the environment with arbitrary target point
target_point = np.random.rand(128)
env = LatentSpaceEnv(action_df, target_point)

# Initialize the DQN model
model = SimpleDQN(env.observation_space, env.action_space)


In [ ]:
num_episodes = 100
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = model.choose_action(state)
        new_state, reward, done, _ = env.step(action)
        model.learn(state, action, reward, new_state)
        state = new_state
        total_reward += reward

    print(f"Episode: {episode}, Total Reward: {total_reward}")